In [ ]:
#import
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from scipy.ndimage import gaussian_filter1d
from scipy import stats
from scipy.stats import binned_statistic
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
#load data
folder = '/Volumes/Elements/data_for_uploaded/'
all_Fres =np.load(folder +'Fig.5/'+'responsive_dFovF.npy',allow_pickle=True) # responses from all layers

#binned data, max bins, each session
bin_con=[]
bin_ch=[]
bins = 150

for session in range(3):
    con = all_Fres[session]
    bin_means, bin_edges, binnumber = binned_statistic(np.arange(0,300),con, statistic='mean', bins= bins)
    bin_con.append(bin_means)
    
    ch = all_Fres[session+3]
    bin_means, bin_edges, binnumber = binned_statistic(np.arange(0,300),ch, statistic='mean', bins= bins)
    bin_ch.append(bin_means)

#concatenate response
bin_min =40
bin_max = 70
bin_conc_ch =np.hstack([np.array(bin_ch)[0][:,bin_min:bin_max],np.array(bin_ch)[1][:,bin_min:bin_max],np.array(bin_ch)[2][:,bin_min:bin_max]])
bin_conc_con = np.hstack([np.array(bin_con)[0][:,bin_min:bin_max],np.array(bin_con)[1][:,bin_min:bin_max],np.array(bin_con)[2][:,bin_min:bin_max]])
bin_all = np.concatenate([bin_conc_con,bin_conc_ch])

#Scaling
mm = MinMaxScaler()
bin_sc = mm.fit_transform(bin_all.T).T

In [ ]:
#PCA
n_components = 4
pca = PCA(n_components=n_components )
bin_p = pca.fit_transform(bin_sc)

In [ ]:
pca.explained_variance_ratio_.sum()

In [ ]:
kmeans_kwargs = {
    "init": "k-means++",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
}
# silhouette coefficients for each k
s_c = []
for k in range(2, 11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(bin_p)
    score = silhouette_score(bin_all , kmeans.labels_)
    s_c.append(score)
plt.plot(range(2, 11), s_c)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
#perform k means clustering
n_clusters=2
kmeans = KMeans(
    init="k-means++",
    n_clusters=n_clusters,
    n_init=1000,
    max_iter=500,
    random_state=42
)

kmeans.fit(bin_p)

cl=kmeans.labels_
cl_con = cl[0:int(len(bin_con[0]))]
cl_ch = cl[int(len(bin_con[0])):]

In [ ]:
c0 = np.array(bin_con)[0][cl_con ==0].mean(axis=0)[50:100].sum()
c1=np.array(bin_con)[1][cl_con ==0].mean(axis=0)[50:100].sum()
if c0>c1:
    clus_act = 0
    clus_inact =1
else:
    clus_act = 1
    clus_inact =0

In [ ]:
#plot heatmap from 100 cells
hmax = 0.08
hmin =-0.08
fig = plt.figure(#figsize=(7*cm, 4*cm)
)
cbar = [False,False,True]
cbar_ax = fig.add_axes([.91, .3, .03, .4])
yticklabels = [50,False,False]
ylabel=['neuron ID',None,None]
fig.subplots_adjust(left=None, bottom=0.1, right=None, top=None, wspace=0.1, hspace=0)
for session in range(3):
    ax = fig.add_subplot(1,3,session+1)
    plt.title('control-session'+str(session+1))
    mask = np.zeros_like(np.array(bin_con)[session], dtype=np.bool)
    mask[:,49] = True
    sns.heatmap(np.concatenate([np.array(bin_con)[session][cl_con ==clus_act][10:75],np.array(bin_con)[session][cl_con ==clus_inact][:35]]),cmap="icefire",vmax=hmax,vmin=hmin,cbar=cbar[session],cbar_ax=cbar_ax,yticklabels =yticklabels[session])
    ax.tick_params(left=False, bottom=False)
    ax.set(xlabel='sec. from vF')
    ax.set_xticks([43,51,60,97])
    ax.set_xticklabels([-1,0,1,5]#,fontsize=fsize
                      )
    ax.set(xlim = [40,100])
    ax.set(ylabel=ylabel[session])
    ax.set_yticklabels(np.arange(0,350,50)#,fontsize=fsize
                      )
    ax.axhline(65,color = 'white',linestyle='-',linewidth=2.5)
    ax.axvline(50,color = 'white',linestyle='-',linewidth=1)
    ax.axvline(60,color = 'white',linestyle='--',linewidth=1)

fig = plt.figure(#figsize=(7*cm, 4*cm)
)
cbar_ax = fig.add_axes([.91, .3, .03, .4])
fig.subplots_adjust(left=None, bottom=0.1, right=None, top=None, wspace=0.1, hspace=0)
for session in range(3):
    ax = fig.add_subplot(1,3,session+1)
    plt.title('Chrim-session'+str(session+1))
    sns.heatmap(np.concatenate([np.array(bin_ch)[session][cl_ch ==clus_act][10:75],np.array(bin_ch)[session][cl_ch ==clus_inact][10:45]]),cmap="icefire",vmax=hmax,vmin=hmin,cbar=cbar[session],cbar_ax=cbar_ax,yticklabels =yticklabels[session])
    ax.tick_params(left=False, bottom=False)
    ax.set(xlabel='sec. from vF')
    ax.set_xticks([43,51,60,97])
    ax.set_xticklabels([-1,0,1,5]#,fontsize=fsize
                      )
    ax.set(xlim = [40,100])
    ax.set(ylabel=ylabel[session])
    ax.set_yticklabels(np.arange(0,350,50)#,fontsize=fsize
                      )
    ax.axhline(65,color = 'white',linestyle='-',linewidth=2.5)
    ax.axvline(50,color = 'white',linestyle='-',linewidth=1)
    ax.axvline(60,color = 'white',linestyle='--',linewidth=1)
